# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings

warnings.filterwarnings('ignore')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

llm_model = "gpt-4.1-nano"

In [9]:
import pandas as pd
df = pd.read_csv('assets/Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [14]:
model = ChatOpenAI(
    model=llm_model,
    temperature=0.7
)

In [17]:
# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "What is the best company name for a company that makes {product}?"
)
# Create a chain
chain = prompt | model | StrOutputParser()
# Test the chain
result = chain.invoke({"product": "colorful socks"})
print(result)

Here are some creative and catchy company name ideas for a colorful sock brand:

1. **Sock Spectrum**  
2. **ColorPop Socks**  
3. **Vivid Toes**  
4. **Rainbow Steps**  
5. **BrightStride**  
6. **Chromatic Socks**  
7. **Hue & Sole**  
8. **ColorSplash Socks**  
9. **KaleidoSocks**  
10. **Prism Socks**  

When choosing the best name, consider factors like uniqueness, availability of domain names, and how well it reflects your brand’s personality.


### Sequential Chains (Single Input → Single Output)

In [18]:
name_prompt = ChatPromptTemplate.from_template(
    "Suggest a company name for a business that makes {product}."
)

name_chain = name_prompt | model | StrOutputParser()

In [19]:
description_prompt = ChatPromptTemplate.from_template(
    "Write a 20-word description for a company called {company_name}."
)

description_chain = description_prompt | model | StrOutputParser()

In [20]:
from langchain_core.runnables import RunnableLambda

overall_chain = (
    name_chain
    | RunnableLambda(lambda name: {"company_name": name})
    | description_chain
)

In [21]:
result = overall_chain.invoke({"product": "queen-size sheet set"})
print(result)

Sure! Sure! offers luxurious, regal queen-size sheet sets designed for ultimate comfort and elegant sleep experiences.


### 3️⃣ Multi-Input / Multi-Output Chains

In [22]:
translate_prompt = ChatPromptTemplate.from_template(
    "Translate this review to English:\n\n{review}"
)

translate_chain = translate_prompt | model | StrOutputParser()

In [23]:
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize this review in one sentence:\n\n{english_review}"
)

summary_chain = summary_prompt | model | StrOutputParser()

In [24]:
language_prompt = ChatPromptTemplate.from_template(
    "What language is this review written in?\n\n{review}"
)

language_chain = language_prompt | model | StrOutputParser()

Run multiple chains in parallel

In [ ]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

initial_step = RunnableParallel(
    english_review=translate_chain,
    language=language_chain,
)

Add summary step

In [31]:
full_chain = (
    initial_step
    | RunnableParallel(
        english_review=itemgetter("english_review"),
        language=itemgetter("language"),
        summary=summary_chain
    )
)
result = full_chain.invoke({"review": df.Review[5]})
print(result)

{'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones from stores and the flavor is much better... Old batch or counterfeit!?", 'language': 'The review is written in French.', 'summary': 'The reviewer finds the taste mediocre and the foam unsatisfactory, suspecting it may be an old or counterfeit batch compared to store-bought versions.'}


In [30]:
followup_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up response to this summary in {language}:\n\n{summary}"
)

followup_chain = followup_prompt | model | StrOutputParser()

final_chain = full_chain | followup_chain
result = final_chain.invoke({"review": df.Review[5]})
print(result)

Je vous remercie pour votre retour. Il est regrettable que la saveur et la mousse n’aient pas été à la hauteur de vos attentes. Nous prenons ces remarques très au sérieux et allons immédiatement vérifier la qualité de nos fournisseurs pour nous assurer de la fraîcheur et de l’authenticité de nos produits. Nous espérons pouvoir améliorer notre offre et vous satisfaire lors de votre prochaine visite.
